# PRÁCTICA 2 Parte II y III

Javier García Serrano, Ana Gil Molina

## Librerías necesarias

In [ ]:
!pip install praw
!pip install rouge_score
!pip install datasets
!pip install trl==0.12.0
!pip install --upgrade bitsandbytes

In [2]:
import os
import json
import praw
import re
import sys

import torch
import textwrap

import nltk
from nltk.tokenize import word_tokenize
from nltk.translate.meteor_score import meteor_score

from rouge_score import rouge_scorer
from sklearn.model_selection import train_test_split

from transformers import TrainingArguments

from datasets import Dataset

import argparse
import torch
from transformers import TrainingArguments
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training, TaskType
from tqdm import tqdm
from trl import SFTTrainer
from trl.trainer import ConstantLengthDataset
from accelerate import Accelerator

In [3]:
# Desactivamos esta herramienta de monitorización que viene configurada por defecto en transformers
os.environ['WANDB_DISABLED'] = 'True'

In [4]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
# Ahora vamos a ver si nuestro entorno tiene GPU o no
device = torch.device ("cuda:0" if torch.cuda.is_available () else "cpu")
print(device)

## <div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>6) Question answering usando instructed fine-tuning (calificación 2)</strong></div>

#### Preparación de los datos

En este ejercicio se pide usar *instructed fine-tuning* para entrenar un modelo capaz de responder preguntas sobre un tema escogido. El *instructed fine-tuning* permite que se le puedan proporcionar instrucciones al modelo acerca de la tarea que debe realizar, mejorando así su capacidad de entender y de responder correctamente, generando respuestas más relevantes para la tarea en cuestión. Para ello, primero es necesario tener un dataset de preguntas, el cual vamos a extraer de Reddit. Hemos escogido el subreddit **r/askscience**, donde se pueden encontrar hilos que hacen preguntas sobre ciencia, y seleccionamos como respuestas válidas los comentarios con mayor puntuación. Reutilizamos el código del ejercicio 1.

In [ ]:
# Ruta base para guardar los archivos
ruta_base = os.path.join(os.getcwd(), 'Question Threads')

# Crear la carpeta si no existe
if not os.path.exists(ruta_base):
    os.makedirs(ruta_base)

# Credenciales de la API de Reddit
reddit = praw.Reddit(client_id='Xn-b5v-4Xof1H5CsGCOF_g',
                     client_secret='rvY41h6nkC53avQB2blMmGexMInyJQ',
                     user_agent='script:DLpPLN-Murcia-2025-AyJ (by u/malatuni99)')

# Subreddits de donde se obtendrán los hilos
subreddit_name = 'askscience'

# Función para limpiar nombres de archivos
def limpiar_nombre_archivo(nombre):
    return re.sub(r'[<>:"/\\|?*]', '_', nombre)[:50]  # Reemplaza caracteres no válidos y limita longitud

A la hora de definir la función `get_threads` para obtener los hilos y, de este modo, construir el dataset, tomamos como referencia el formato del dataset ALPACA. Este dataset consta de una columna llamada `instructions`, que contiene las instrucciones sobre lo que debe hacer el modelo (en nuestro caso, la pregunta); una columna `input`, que incluye información adicional que se le puede proporcionar al modelo (en este caso, el texto asociado al hilo); y por último, una columna `output`, que contiene una respuesta válida de referencia (en nuestro caso, los comentarios del hilo con mayor puntuación).

In [ ]:
def get_threads(subreddit_name, limit, max_comments):
    subreddit = reddit.subreddit(subreddit_name)
    posts = subreddit.top(limit=limit)
    results = []

    # Moderadores a excluir
    excluded_moderators = ['MockDeath', 'AskScienceModerator']

    for post in posts:
        # Excluir los hilos creados por los moderadores
        if post.author and post.author.name in excluded_moderators:
            continue  # Saltar este hilo y no agregarlo a los resultados

        post.comments.replace_more(limit=0)
        comments = post.comments.list()

        # Seleccionar los comentarios con más votos
        top_comments = sorted(comments, key=lambda x: x.score, reverse=True)[:max_comments]

        # Extraer el texto de los comentarios válidos
        responses = [comment.body for comment in top_comments if comment.body]

        # Agregar el hilo al dataset creando instancias separadas por cada respuesta
        for response in responses:
            results.append({
                'instruction': post.title,
                'input': post.selftext if post.selftext else None,
                'output': response
            })

    return results

Vamos a compilar un dataset de 500 preguntas, y para cada una, tomamos los 3 comentarios con más votos. De esta forma, el dataset tendrá unas 1500 instancias (excluyendo aquellos hilos escritos por moderadores, dado que no suelen ser preguntas).

In [ ]:
# Descargar y guardar los hilos
threads = get_threads(subreddit_name, limit=500, max_comments=3)

for idx, post in enumerate(threads, start=1):
    safe_title = limpiar_nombre_archivo(post['instruction'])  # Limpiar el título para usarlo como nombre de archivo
    archivo_path = os.path.join(ruta_base, f'{safe_title}_thread_{idx}.json')

    with open(archivo_path, 'w', encoding='utf-8') as f:
        json.dump(post, f, indent=4, ensure_ascii=False)

# Mostrar el número total de instancias
print(f"Se han cargado un total de {len(threads)} instancias.")

Ahora cargamos los datos almacenados en la carpeta "Question Threads".

In [ ]:
# Ruta de la carpeta base 'Question Threads'
script_dir = os.getcwd()                                    # Ruta actual del notebook
threads_dir = os.path.join(script_dir, 'Question Threads')  # Ruta a la carpeta 'Question Threads'

# Listar todos los archivos JSON en la carpeta 'Question Threads'
json_files = [f for f in os.listdir(threads_dir)]

print("Archivos JSON encontrados:", json_files)

Creamos un dataset a partir de los datos que acabamos de cargar. A la hora de añadir una instancia al dataset, comprobamos que la respuesta asociada no haya sido eliminada.

In [ ]:
# Lista para almacenar los datos cargados
dataset = []

# Leer cada archivo JSON y cargar sus datos
for json_file in json_files:
    file_path = os.path.join(threads_dir, json_file)
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

        if (data['output'] != '[removed]') and (data['output'] != '[deleted]'):
            dataset.append(data)

# Mostrar la cantidad de instancias cargadas y un ejemplo
print(f"Se han cargado {len(dataset)} instancias.\n")
print("Ejemplo de una instancia:", json.dumps(dataset[0], indent=4, ensure_ascii=False))

Dado que debemos reentrenar el modelo, y luego evaluarlo, necesitamos un conjunto de entrenamiento, uno de validación y otro de prueba. Por ello, dividimos el dataset en estos tres conjuntos, tomando una proporción del $80\%$ para entrenamiento, $10\%$ para validación y $10\%$ para prueba.

In [ ]:
# Dividir el dataset en entrenamiento (80%), validación (10%) y prueba (10%)
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
eval_dataset, test_dataset = train_test_split(test_dataset, test_size=0.5, random_state=42)

print(f"Entrenamiento: {len(train_dataset)} instancias")
print(f"Validación: {len(eval_dataset)} instancias")
print(f"Prueba: {len(test_dataset)} instancias")

Además, convertimos estos tres subconjuntos de datos en Datasets de Hugging Face, para poder trabajar con ellos de forma más eficiente.

In [9]:
# Convertir los subconjuntos de datos en objetos Dataset de Hugging Face
train_dataset = Dataset.from_list(train_dataset)
eval_dataset = Dataset.from_list(eval_dataset)
test_dataset = Dataset.from_list(test_dataset)

#### Entrenamiento del modelo Gemma (2B)

Ahora que ya tenemos el dataset, pasamos a cargar el modelo. En este caso, vamos a usar el modelo **Gemma (2B)**. Gemma es una familia de modelos de tipo *text-to-text*, y son adecuados para distintas tareas de generación de texto, como la respuesta a preguntas, resúmenes y razonamiento.

In [ ]:
from huggingface_hub import login

login("")   # Introducir token de acceso de Hugging Face

In [ ]:
# Cargar el tokenizador de Gemma 2B
model_path = "google/gemma-2-2b"
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)

Con todo esto, ahora definimos la función `prepare_sample_text`, encargada de preprocesar los datos que se emplearán durante el entrenamiento y la validación del modelo. Su finalidad es estructurar la información contenida en un ejemplo del conjunto de datos para que el modelo entienda cómo interpretar las instrucciones, el contexto (si lo hay), y la respuesta asociada. Para ello, se genera un texto cuyo formato está estructurado con etiquetas claras que separan dichos elementos del ejemplo.

Los textos generados que recibirá el modelo comenzarán con una instrucción acerca de lo que se espera que haga, en este caso, que responda a la pregunta proporcionada de forma detallada y con una explicación científica. A continuación, se proporcionará un cierto contexto (esto es, el texto del hilo correspondiente, si lo hay), para de esta forma proporcionar más contexto sobre la pregunta, y por último se añadirá la respuesta de referencia.

In [12]:
def prepare_sample_text(example):
    """
    Prepare the text from a sample of the dataset by formatting it
    according to the instruction and input.

    Args:
        example: A dictionary containing 'instruction', 'input', and 'output'.

    Returns:
        A formatted string combining the instruction, input, and output.
    """

    if example['input'] == None:
        text = f"### Answer the next question in detail with scientific explanations:\n{example['instruction']}\n\n### Response:\n{example['output']}"
    else:
        text = f"### Answer the next question in detail with scientific explanations:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n{example['output']}"

    return text

Ahora, gracias a la función `prepare_sample_text` que acabamos de definir, podemos añadir una nueva función `chars_token_ratio` para calcular la relación promedio entre el número de caracteres y el número de tokens en un conjunto de datos. Este valor nos será útil para ajustar parámetros relacionados con el tamaño de los datos procesados durante el entrenamiento del modelo.

In [13]:
def chars_token_ratio(dataset, tokenizer, nb_examples=400):
    """
    Estimate the average number of characters per token in the dataset.

    Args:
        dataset: The dataset to analyze.
        tokenizer: The tokenizer to use for tokenizing text.
        nb_examples: Number of examples to consider for estimation.

    Returns:
        The average number of characters per token.
    """

    total_characters, total_tokens = 0, 0

    # Iterate over the dataset samples
    for _, example in tqdm(zip(range(nb_examples), iter(dataset)), total=nb_examples):
        text = prepare_sample_text(example)
        total_characters += len(text)

        # Tokenize the text and count the tokens
        if tokenizer.is_fast:
            total_tokens += len(tokenizer(text).tokens())
        else:
            total_tokens += len(tokenizer.tokenize(text))

    # Calculate and return the character-to-token ratio
    return total_characters / total_tokens

A continuación, definimos la función `train_model`, utilizada para entrenar un modelo de lenguaje pre-entrenado mediante *fine-tuning*, utilizando una configuración LoRA para optimizar los recursos de entrenamiento. Los parámetros para el entrenamiento se definen usando `TrainingArguments`, donde se especifican, entre otros, los siguientes parámetros:

- `output_dir`: carpeta para guardar los resultados.

- `evaluation_strategy`: se evalúa el modelo al final de cada época.

- `learning_rate`: tasa de aprendizaje.

- `batch_size`: tamaño del lote para entrenamiento y para validación.

- `num_train_epochs`: número de épocas, esto es, de pasadas completas por el conjunto de entrenamiento.

- `weight_decay`: regularización para evitar sobreajuste.

Además, se utiliza el trainer personalizado `SFTTrainer` que permite aplicar *packing* para mejorar la eficiencia.

In [14]:
def train_model(model, train_dataset, eval_dataset, lora_config, save_path):
    """
    Train the model using the provided training and evaluation datasets,
    LoRA configuration, and save the trained model to the specified path.

    Args:
        model: The pre-trained language model to fine-tune.
        train_dataset: The dataset used for training.
        eval_dataset: The dataset used for evaluation.
        lora_config: The LoRA configuration for parameter-efficient fine-tuning.
        save_path: The directory path where the trained model will be saved.
    """

    # Set hyperparameters for training
    batch_train_size = 1
    batch_eval_size = 1
    EPOCHS = 1

    # Define training arguments
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=EPOCHS,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=1,
        per_device_train_batch_size=batch_train_size,
        per_device_eval_batch_size=batch_eval_size,
        fp16=True,  # Use 16-bit (mixed) precision
        weight_decay=0.05,
        learning_rate=1e-4,
        warmup_steps=100,
        max_steps=500
    )

    # Initialize the SFT (Supervised Fine-Tuning) Trainer
    trainer = SFTTrainer(
        model=model,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        peft_config=lora_config,
        packing=True,
        max_seq_length=None,
        tokenizer=tokenizer,
        args=training_args,
    )

    # Start training
    trainer.train()

    # Save the fine-tuned model
    trainer.save_model(f'{save_path}/Gemma')
    trainer.model.save_pretrained(f'{save_path}/final_checkpoint')

Por último, implementamos la función `main`, encargada de organizar los distintos pasos para llevar a cabo el *fine-tuning* del modelo. En primer lugar, se utiliza la función `chars_token_ratio` para calcular el número promedio de caracteres por token. A continuación, se utiliza dicho número, junto con la función `prepare_sample_text`, para crear datasets de longitud constante para entrenamiento y validación. Además, se utiliza cuantización en 4 bits para así poder manejar el modelo en GPUs con memoria limitada. Después, se carga el modelo pre-entrenado con cuantización en 4 bits, se desactiva el caché para evitar problemas de memoria, y se definen algunos parámetros para el *fine-tuning* eficiente mediante LoRA. Finalmente, se prepara el modelo para el entrenamiento con cuantización, se entrena y se guarda en la ruta especificada.

In [15]:
def main(args, train_data, eval_data):
    """
    The main function to orchestrate dataset loading, model preparation,
    training, and saving.

    Args:
        args: Command-line arguments parsed by argparse.
        train_data: Preloaded training dataset.
        eval_data: Preloaded validation dataset.
    """

    save_path = args.save_path

    # Estimate the average number of characters per token
    chars_per_token = chars_token_ratio(train_data, tokenizer)

    # Create a constant length dataset for training
    train_dataset = ConstantLengthDataset(
        tokenizer,
        train_data,
        formatting_func=prepare_sample_text,
        infinite=True,
        seq_length=512,
        chars_per_token=chars_per_token,
    )

    # Create a constant length dataset for validation
    eval_dataset = ConstantLengthDataset(
        tokenizer,
        eval_data,
        formatting_func=prepare_sample_text,
        infinite=False,
        seq_length=512,
        chars_per_token=chars_per_token,
    )

    # Configuration for 4-bit quantization using BitsAndBytes
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    # Load the pre-trained model with 4-bit quantization
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        quantization_config=bnb_config,
        device_map={"": Accelerator().local_process_index},
        trust_remote_code=True,
        use_auth_token=True,
    )

    # Disable caching to prevent memory issues during fine-tuning
    model.config.use_cache = False

    # Configure LoRA (Low-Rank Adaptation) for parameter-efficient fine-tuning
    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type=TaskType.CAUSAL_LM
    )

    # Prepare the model for k-bit (quantized) training
    model = prepare_model_for_kbit_training(model)

    # Train the model using the training and validation datasets
    train_model(model, train_dataset, eval_dataset, lora_config, save_path)

Ahora que ya tenemos las funciones necesarias, pasamos a entrenar y guardar el modelo.

In [ ]:
torch.cuda.empty_cache()

if __name__ == '__main__':
    sys.argv = [sys.argv[0]]

    # Parse command-line arguments
    parser = argparse.ArgumentParser()
    parser.add_argument('--save_path', type=str, default='./results', help='Directory to save the trained model')
    args = parser.parse_args()

    # Call the main function with parsed arguments
    main(args, train_dataset, eval_dataset)

#### Evaluación del modelo

Una vez realizado el instructed fine-tuning del modelo Gemma (2B), pasamos a evaluar dicho modelo en el conjunto de prueba que habíamos guardado anteriormente. Comenzamos cargando el modelo y el tokenizador.

In [ ]:
# Ruta del modelo guardado
model_path = os.path.join(os.getcwd(), 'results', 'Gemma')

# Cargar el modelo
model = AutoModelForCausalLM.from_pretrained(model_path)

# Mover el modelo al dispositivo disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Cargar el tokenizador
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)

Análogamente a como se hacía con los datos de entrenamiento y de validación, ahora se define una función `prepare_sample_text_test`, encargada de preprocesar los ejemplos de test con un formato que utiliza etiquetas que indican las instrucciones sobre cómo responder a la pregunta proporcionada, el posible contexto adicional, y una etiqueta indicando que se espera una respuesta. Sin embargo, a diferencia de la función utilizada para los datos de entrenamiento, esta nueva función no incluye el contenido de la respuesta en el texto formateado, pues se espera que dicha respuesta la proporcione el modelo en la predicción generada.

In [20]:
def prepare_sample_text_test(example):
    """
    Prepare the text from a sample of the dataset by formatting it
    according to the instruction and input.

    Args:
        example: A dictionary containing 'instruction', 'input', and 'output'.

    Returns:
        A formatted string combining the instruction, input, and output.
    """

    if example['input'] == None:
        text = f"### Answer the next question in detail with scientific explanations:\n{example['instruction']}\n\n### Response:"
    else:
        text = f"### Answer the next question in detail with scientific explanations:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:"

    return text

Con todo esto, pasamos a realizar las predicciones sobre el conjunto de test. El parámetro `max_new_tokens` controla el número máximo de tokens que el modelo generará para la salida. Dado que nuestros datos provienen del subreddit r/askscience, donde las respuestas suelen ser complejas y detalladas, tomamos un valor de $100$ para que el modelo pueda proporcionar respuestas relativamente largas y bien estructuradas, pero sin ser excesivamente largas.

In [ ]:
predictions = []

# Poner el modelo en modo evaluación
model.eval()

# Iterar sobre el conjunto de test
with torch.no_grad():
    for example in test_dataset:
        # Preprocesar el ejemplo
        input_text = prepare_sample_text_test(example)
        input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

        # Generar las predicciones
        output = model.generate(input_ids, max_new_tokens=100, num_return_sequences=1, no_repeat_ngram_size=2)

        # Decodificar y almacenar la predicción
        decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
        predictions.append(decoded_output)

Para evaluar los resultados, comenzaremos analizando algunos ejemplos de manera manual. Para ello, seleccionaremos varios hilos del conjunto de test, y mostraremos la pregunta correspondiente junto con la respuesta de referencia. A continuación, mostraremos la respuesta generada por el modelo y evaluaremos si esta tiene sentido y es coherente.

In [ ]:
# Configurar el ancho máximo de cada línea
max_width = 140

print('-Ejemplo 1-')
print('Pregunta:')
print(textwrap.fill(test_dataset[0]['instruction'], width=max_width))

print('\nRespuesta de referencia:')
print(textwrap.fill(test_dataset[0]['output'], width=max_width))

print('\nRespuesta generada por el modelo:')
print(textwrap.fill(predictions[0], width=max_width))

Para este ejemplo, se pregunta cómo una red informática, como la de HBO, gestiona la enorme cantidad de datos que debe transmitir en cortos períodos de tiempo, por ejemplo, cuando millones de personas intentan ver el mismo contenido al mismo tiempo.

Observamos que la respuesta generada por el modelo incluye la pregunta y el contexto adicional que se le habían proporcionado, lo cual no era necesario, y podría indicar una cierta confusión del modelo con los datos de entrada. Centrándonos específicamente en la respuesta a dicha pregunta, esta sugiere que una solución sería usar una gran cantidad de servidores dedicados a transmitir el contenido simultáneamente a todos los usuarios. De esta forma, el modelo proporciona un razonamiento que pretende responder a la pregunta, basándose en la idea de requerir infraestructura para manejar grandes volúmenes de datos.

Esta respuesta se alinea correctamente con la pregunta. Sin embargo, la explicación es algo vaga y carece de profundidad científica, aunque esto podría deberse a que los datos de entrenamiento, extraídos de Reddit, podrían no ser tampoco demasiado técnicos.

In [ ]:
print('-Ejemplo 2-')
print('Pregunta:')
print(textwrap.fill(test_dataset[1]['instruction'], width=max_width))

print('\nRespuesta de referencia:')
print(textwrap.fill(test_dataset[1]['output'], width=max_width))

print('\nRespuesta generada por el modelo:')
print(textwrap.fill(predictions[1], width=max_width))

En este segundo ejemplo, se pregunta por el impacto de una tormenta como el huracán Harvey en el paisaje y los ecosistemas de las áreas afectadas. Además, se busca saber cómo se retira el agua acumulada después de un evento catastrófico como Harvey, que generó grandes inundaciones. Por último, se pregunta si se crearán ríos o lagos permanentes.

De nuevo, la respuesta generada por el modelo incluye la pregunta proporcionada, pero centrándonos específicamente en la respuesta a dicha pregunta, el modelo indica en primer lugar que el agua retrocederá del suelo, sin entrar en detalles acerca de los procesos que darían lugar a dicho acontecimiento. Sobre los cambios en el terreno, indica que el suelo cambiará permanentemente, pues habrá una saturación con agua salada (según ChatGPT, esto sería más relevante para una inundación cerca de áreas costeras donde el agua de mar está involucrada, pero Harvey fue mayormente una tormenta de agua dulce) y además quedarán escombros en el suelo (según ChatGPT, esto es correcto, ya que las tormentas suelen dejar una cantidad significativa de desechos, sedimentos y otros materiales en las áreas afectadas). Finalmente, no se responde acerca de si se formarían nuevos ríos o lagos permanentes.

En general, gracias a estos dos ejemplos considerados, hemos visto que las respuestas generadas por el modelo suelen mantener cierta relevancia con respecto a la pregunta original. El modelo intenta abordar el tema principal planteado en la pregunta, lo que indica que tiene un entendimiento básico del contexto. Por ejemplo, en el primer ejemplo, mencionó el uso de servidores para manejar datos, y en el segundo, reconoció que las tormentas pueden cambiar el terreno y dejar escombros.

Aunque las respuestas generadas están relacionadas con las preguntas, carecen de profundidad o precisión técnica en muchos casos. El modelo tiende a generalizar y omitir detalles importantes, como los procesos específicos involucrados en el drenaje del agua o la gestión de redes de datos masivos. Sin embargo, esto podría deberse a que los datos usados en el entrenamiento podrían ser algo vagos, al haber sido extraídos de Reddit, en lugar de un paper científico o alguna revista más técnica.

En algunos casos, el modelo introduce elementos que no son del todo precisos o están fuera de contexto. Por ejemplo, en el segundo ejemplo, mencionó que el suelo estaría saturado de agua salada, lo cual no es aplicable a las áreas afectadas por Harvey, que estuvieron mayormente inundadas con agua dulce.

Además, el modelo a menudo omite partes clave de las preguntas, como en el segundo ejemplo, donde no respondió si podrían formarse ríos o lagos permanentes.

Por último, la estructura de las respuestas requiere un post-procesamiento, ya que incluye elementos que no son estrictamente necesarios. Por ejemplo, las etiquetas como `### Answer the next question in detail with scientific explanations:`, `### Input:` o `### Response:`, que se utilizan para estructurar los datos durante el entrenamiento y guiar al modelo, pero que no aportan valor a la evaluación de las respuestas y necesitan ser eliminadas. También, en las respuestas generadas, el modelo suele incluir nuevamente la pregunta original y el contexto (si es el caso) dentro del texto, lo cual no es necesario, pues introduce redundancia y complica la interpretación directa de la respuesta. Por esta razón, se introduce la función `clean_response` encargada de limpiar las respuestas, eliminando todo lo que no nos interesa.

In [ ]:
def clean_response(text):
    """
    Limpia la respuesta generada por el modelo, eliminando todo lo que viene antes de '### Response:'
    y dejando solo la parte de la respuesta.

    Args:
        text (str): La respuesta generada por el modelo que contiene la etiqueta '### Response:'

    Returns:
        str: La respuesta limpia.
    """
    # Buscar la parte del texto después de '### Response:' y devolverla
    match = re.search(r"### Response:(.*)", text, re.DOTALL)

    if match:
        # Limpiar espacios en blanco al inicio y final de la respuesta
        return match.group(1).strip()
    else:
        # Si no se encuentra la etiqueta '### Response:', devolver el texto original
        return text.strip()

# Ejemplo de uso (ejemplo 1 anterior):
response = predictions[0]

cleaned_response = clean_response(response)
print(cleaned_response)

In [ ]:
cleaned_predictions = []

for response in predictions:
    cleaned_response = clean_response(response)
    cleaned_predictions.append(cleaned_response)

# Ejemplo
print(cleaned_predictions[0])

Para evaluar el modelo de manera más objetiva, ahora vamos a utilizar diferentes métricas para cuantificar la calidad de las respuestas generadas. Algunas métricas comunes en tareas de generación de texto que podemos usar para evaluar el modelo son:

1. ROUGE-N: Esta métrica compara la superposición de n-gramas (secuencias de n palabras consecutivas) entre las predicciones generadas por el modelo y las respuestas de referencia. Por ejemplo, con ROUGE-1 se comparan unigramas, esto es, palabras individuales, mientras que con ROUGE-2 se comparan bigramas, es decir, pares de palabras consecutivas. Además, ROUGE-L evalúa la longitud de la subsecuencia común más larga entre la respuesta generada y la respuesta de referencia.

2. METEOR: Esta métrica evalúa la similitud entre las respuestas generadas y las de referencia tomando en cuenta coincidencias exactas de palabras, sinónimos, raíces y el orden de las palabras. Además, prioriza el equilibrio entre precisión y recall.

Comencemos con ROUGE-N. A continuación, se define una función que permite calcular la métrica ROUGE-N para evaluar la calidad de las respuestas generadas en comparación con las respuestas de referencia. Utiliza el paquete `rouge_scorer` para obtener las métricas ROUGE-1, ROUGE-2 y ROUGE-L. Para ello, recorre las listas de predicciones y referencias, y para cada par, va calculando las métricas. Finalmente, se calcula el promedio de cada métrica y se devuelve como un diccionario.

In [35]:
# Función para calcular ROUGE-N
def compute_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    scores = {"rouge1": [], "rouge2": [], "rougeL": []}

    # Recorrer todas las predicciones y referencias
    for pred, ref in zip(predictions, references):
        score = scorer.score(ref, pred)
        for key in scores:
            scores[key].append(score[key].fmeasure)

    # Promediar las puntuaciones ROUGE
    avg_scores = {key: sum(value) / len(value) for key, value in scores.items()}
    return avg_scores

In [36]:
# Obtener las respuestas de referencia del dataset
test_references = [test_example['output'] for test_example in test_dataset]

In [ ]:
# Evaluar ROUGE-N
rouge_scores = compute_rouge(cleaned_predictions, test_references)

print("ROUGE-1:", rouge_scores["rouge1"])
print("ROUGE-2:", rouge_scores["rouge2"])
print("ROUGE-L:", rouge_scores["rougeL"])

Un valor de $0.1964$ para el ROUGE-1 indica que, en promedio, alrededor del $19.64\%$ de las palabras en las respuestas generadas coinciden con las palabras en las respuestas de referencia. Aunque esto sugiere que el modelo logra captar ciertas palabras clave del contenido, la coincidencia es baja, lo que podría indicar que las respuestas generadas no están suficientemente alineadas con las de referencia en términos de vocabulario relevante.

Un valor de $0.0240$ para ROUGE-2 es significativamente menor que el de ROUGE-1, lo que sugiere que el modelo tiene dificultades para captar relaciones contextuales más complejas entre pares de palabras consecutivas en las respuestas de referencia.

Por último, un valor de $0.1188$ para ROUGE-L sugiere que, aunque el modelo puede captar algunas palabras clave, las frases completas o las ideas principales de las respuestas tienden a estar algo peor alineadas, en relación con las respuestas de referencia.

Ahora, vamos a calcular la métrica METEOR sobre nuestros datos de test. Para ello, definimos una función `compute_meteor`, que recorre las listas de predicciones y referencias, y para cada par, calcula la métrica METEOR. Finalmente, se calcula el promedio de la métrica y se devuelve su valor.

In [ ]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt_tab')

In [38]:
# Función para calcular METEOR
def compute_meteor(predictions, references):
    scores = []

    for pred, ref in zip(predictions, references):
        # Tokenizar las predicciones y las referencias
        pred_tokens = word_tokenize(pred)
        ref_tokens = word_tokenize(ref)

        # Calcular METEOR para cada par de predicción y referencia
        score = meteor_score([ref_tokens], pred_tokens)
        scores.append(score)

    # Promediar las puntuaciones METEOR
    avg_score = sum(scores) / len(scores) if scores else 0
    return avg_score

In [ ]:
# Evaluar METEOR para las predicciones
meteor_score_value = compute_meteor(cleaned_predictions, test_references)

# Imprimir la puntuación METEOR
print(f"METEOR: {meteor_score_value:.4f}")

Finalmente, un valor de $0.1215$ para METEOR indica una baja alineación semántica entre las respuestas generadas y las de referencia. Este resultado sugiere que, aunque algunas palabras clave o conceptos relevantes están presentes, el modelo tiene dificultades para generar respuestas que coincidan con el significado completo de las respuestas de referencia. Esto podría deberse a que no alcanza un nivel de detalle suficiente, o que genera respuestas que son medianamente válidas (las que hemos revisado a mano no parecían ser del todo incorrectas), pero diferentes a las de referencia.